# 1. Imports

In [1]:
import sys, os, datetime, pickle, time
import string, pdb, tqdm
import random, keras, os.path, yaml
import pandas as pd
import numpy as np
#
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
#
import tensorflow as tf
from tensorflow.keras import models
#
from timeit import default_timer as timer
from datetime import datetime, timedelta, date
from PIL import Image
from IPython.display import Image
# %matplotlib inline

#
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
# config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.get_logger().setLevel('ERROR')

[]


# 2. Preparing Data

In [2]:
# Obtener la ruta del directorio tres niveles arriba
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))

# Cargar datasets desde el directorio tres niveles arriba
data_norm_path = os.path.join(parent_dir, 'Dataset/data_norm.pkl')

with open(data_norm_path, 'rb') as handle:
    GlblFrm = pickle.load(handle)
    dosD    = pickle.load(handle)
    coils   = pickle.load(handle)
    nclmaps = pickle.load(handle)


# 3. Building Sets

In [3]:
#  Selecting coils OK and NOK
lsids1 = coils.loc[coils['Label']==1,'SID'].tolist()
lsids2 = coils.loc[coils['Label']==2,'SID'].tolist()
#
# Select and extract 30 coil's sids for independent assessemnt
ass1 = random.sample(lsids1, 30)
ass2 = random.sample(lsids2, 30)
res1 = np.array(list(set(lsids1) - set(ass1)))
res2 = np.array(list(set(lsids2) - set(ass2)))
#
# The remaining are organized for building the model
random.shuffle(res1)
random.shuffle(res2)
train1, validate1, test1 = np.split(res1,[int(.7*len(res1)), int(.9*len(res1))])
train2, validate2, test2 = np.split(res2,[int(.7*len(res2)), int(.9*len(res2))])
#
# Building the full sets
train = train1.tolist() + train2.tolist()
valid = validate1.tolist() + validate2.tolist()
test  = test1.tolist()  + test2.tolist()
ass   = ass1   + ass2
#

In [4]:
def featureMap(id, nlcmaps):
    arr1 = nclmaps[id][1234]['nzne'].to_numpy()
    arr2 = nclmaps[id][1243]['nzne'].to_numpy()
    arr  = np.concatenate((arr1, arr2), axis=1)
    return(arr)
#
def prep_dataset(setd,nlcmaps):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        setd_f.append(arrimg)
        if lbl == 1:
            setd_l.append([1.,0.])
        if lbl == 2:
            setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])
#
def factory_rep(arr,step=0):
    # arr image havind 264 rows and 18 (9 face A+9 face B) columns of normalized data 
    # mirror per face over x axis
    # channels 0-3 => 5-8 and 5-8 => 0-3
    permut1 = list(range(8,4,-1))+[4] + list(range(3,-1,-1))+ list(range(17,13,-1))+[13]+list(range(12,8,-1))
    idx = np.empty_like(permut1)
    idx[permut1] = np.arange(len(permut1))
    arr1 = arr[:,idx]
    permut2 = list(range(263,131,-1))+ list(range(131,-1,-1))
    idx = np.empty_like(permut2)
    idx[permut2] = np.arange(len(permut2))    
    arr2 = arr[idx,:]
    arr3 = arr1[idx,:]
    res  = [arr, arr1, arr2, arr3]
    if step > 0:
        newa = arr
        end = arr.shape[0]-step
        for i in range(arr.shape[0] // step):
            permut = list(range(arr.shape[0]-step,arr.shape[0]))+list(range(0,end))
            idx = np.empty_like(permut)
            idx[permut] = np.arange(len(permut))
            newb= newa[idx,:]
            res.append(newb)
            newa= newb
    res = np.array(res)
    return(res)
#

def prep_dataset_aug(setd,nlcmaps,tlab=-1):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        if lbl == tlab: # if lower class => higher augmentation
            res = factory_rep(arrimg,step=8)
        else:
            res = factory_rep(arrimg)
        for j in range(res.shape[0]):
            setd_f.append(res[j,:,:])
            if lbl == 1:
                setd_l.append([1.,0.])
            if lbl == 2:
                setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])

In [5]:
goal = 'C:/Users/alexm/OneDrive/Escritorio/UNIVERSIDAD/TFG/ALVAREZ_CASTRO_Alejandro/ALVAREZ_CASTRO_Alejandro/ZN_1D_imgs'
# for i in train + valid + test:
train_f, train_l = prep_dataset_aug(train,nclmaps,2)
np.savez(goal+'train.npz', features=train_f, labels=train_l)
#
valid_f, valid_l = prep_dataset(valid,nclmaps)
np.savez(goal+'validation.npz', features=valid_f, labels=valid_l)
#
test_f, test_l = prep_dataset(test,nclmaps)
np.savez(goal+'test.npz', features=test_f, labels=test_l)
#
ass_f, ass_l = prep_dataset(ass,nclmaps)
np.savez(goal+'assess.npz', features=ass_f, labels=ass_l)
#

In [6]:
#
npzfile = np.load(goal+'train.npz')
print(npzfile['features'].shape)
print(npzfile['labels'].shape)

(4293, 264, 18)
(4293, 2)


# 4. Building and Training the model

In [10]:
# Ruta donde se encuentran los datos y donde se guardarán los archivos de anotación
data_folder = 'C:/Users/alexm/OneDrive/Escritorio/UNIVERSIDAD/TFG/ALVAREZ_CASTRO_Alejandro/ALVAREZ_CASTRO_Alejandro/ZN_1D_imgs'
annotation_folder = 'C:/Users/alexm/OneDrive/Escritorio/UNIVERSIDAD/TFG/ALVAREZ_CASTRO_Alejandro/ALVAREZ_CASTRO_Alejandro/anotation'

# Carga de los archivos .npz que contienen los datos de entrenamiento, validación, prueba y evaluación
train_data = np.load(data_folder + "train.npz")
valid_data = np.load(data_folder + "validation.npz")
test_data = np.load(data_folder + "test.npz")
assess_data = np.load(data_folder + "assess.npz")

assess_data

In [24]:
print("Claves de asses_data: ",list(assess_data.keys()))
print("Dimensiones de features: ",assess_data['features'].shape)
print("Dimensiones de labels: ",assess_data['labels'].shape)

Claves de asses_data:  ['features', 'labels']


AttributeError: 'NpzFile' object has no attribute 'to_dict'

In [ ]:
# Función para escribir la información de anotación en archivos de texto
def write_annotation_file(data, annotation_folder, prefix):
    annotations = data["labels"]
    images = data["features"]
    
    for i, image in enumerate(images):
        annotation = annotations[i]
        
        # Obtener las coordenadas de los cuadros delimitadores y la clase
        # Supongamos que las coordenadas se encuentran en las primeras 4 columnas del array
        bbox_coordinates = annotation[:4]
        label = annotation[4]  # 1 para "OK", 2 para "NOK"
        
        # Escribir las coordenadas y la clase en el archivo de anotación
        annotation_file_path = os.path.join(annotation_folder, f"{prefix}_{i}.txt")
        with open(annotation_file_path, "w") as f:
            f.write(f"{label} {' '.join(map(str, bbox_coordinates))}")
        print(f"Archivo de anotación guardado en {annotation_file_path}")

# Escribir archivos de anotación para los conjuntos de entrenamiento, validación, prueba y evaluación
write_annotation_file(train_data, annotation_folder, "train")
write_annotation_file(valid_data, annotation_folder, "validation")
write_annotation_file(test_data, annotation_folder, "test")
write_annotation_file(assess_data, annotation_folder, "assess")

# Crear archivos de configuración (ejemplo)
# Esto puede variar dependiendo de los detalles específicos de tu configuración y entrenamiento de YOLOv7
config_template = """
[dataset]
train = ruta/a/tus/datos/train.txt
val = ruta/a/tus/datos/validation.txt
test = ruta/a/tus/datos/test.txt
names = ruta/a/tus/datos/classes.names
backup = backup/

[net]
# Tú configuración de red

[training]
# Tú configuración de entrenamiento
"""
config_file_path = "ruta/a/tus/datos/yolov7_custom_config.cfg"
with open(config_file_path, "w") as f:
    f.write(config_template)
print(f"Archivo de configuración guardado en {config_file_path}")

# Organizar la estructura de carpetas
os.makedirs(os.path.join(data_folder, "images"))
os.makedirs(os.path.join(data_folder, "labels"))
os.rename(os.path.join(annotation_folder, "*.txt"), os.path.join(data_folder, "labels"))

print("Estructura de carpetas organizada correctamente.")